# 🚀 视频 OCR (PaddleOCR) GPU vs CPU 速度对比测试

测试 GPU 加速对 PaddleOCR 视频字幕提取的性能提升

## 📋 测试内容
- CPU vs GPU OCR 速度对比
- 不同视频长度性能测试
- 不同置信度阈值效果对比
- 中英文识别效果对比

## 1️⃣ 环境设置

In [ ]:
# 安装依赖（使用旧版本避免 langchain 依赖问题）
!pip install -q paddlepaddle-gpu paddleocr==2.7.0.3 yt-dlp opencv-python

import cv2
import time
import numpy as np
from pathlib import Path
from paddleocr import PaddleOCR

# 检测 GPU
try:
    import paddle
    gpu_available = paddle.is_compiled_with_cuda()
    print(f"🔥 GPU 可用: {gpu_available}")
    if gpu_available:
        print(f"   GPU 设备: {paddle.get_device()}")
        print(f"   GPU 数量: {paddle.device.cuda.device_count()}")
except:
    gpu_available = False
    print("⚠️ Paddle GPU 不可用，将使用 CPU")

## 2️⃣ 下载测试视频

In [ ]:
# 测试视频链接（B站短视频）
TEST_URL = "https://www.bilibili.com/video/BV1GJ411x7h7"  # @param {type:"string"}

import yt_dlp

print("⬇️ 下载测试视频...")
ydl_opts = {
    'format': 'worst[ext=mp4]/worst',  # 最低画质加快下载
    'outtmpl': 'test_video.%(ext)s',
    'quiet': True,
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(TEST_URL, download=True)
    duration = info.get('duration', 0)
    title = info.get('title', 'test')

video_file = list(Path('.').glob('test_video.*'))[0]
print(f"\n✅ 视频: {video_file}")
print(f"📊 时长: {duration}秒 ({duration/60:.1f}分钟)")

In [ ]:
# 流式 OCR 测试（边下边处理，无需等待完整下载）
TEST_URL = "https://www.bilibili.com/video/BV1GJ411x7h7"  # @param {type:"string"}
SAMPLE_FPS = 1  # 每秒采样帧数
CONF_THRESHOLD = 0.5

# 流式处理函数
import subprocess
import threading
import queue

def stream_video_with_ocr(url, ocr, sample_fps=1, conf_threshold=0.5, max_duration=None):
    """
    流式处理：边下载边OCR
    
    Args:
        url: 视频URL
        ocr: PaddleOCR实例
        sample_fps: 采样帧率
        conf_threshold: 置信度阈值
        max_duration: 最大处理时长（秒），None表示全部
    """
    print("\n" + "="*60)
    print("🚀 流式 OCR 处理（边下边OCR）")
    print("="*60)
    
    total_start = time.time()
    all_texts = []
    frame_count = 0
    text_count = 0
    
    # 使用 yt-dlp + ffmpeg 流式读取
    # yt-dlp 获取视频URL
    print("📡 获取视频流地址...")
    ydl_opts = {
        'format': 'worst[ext=mp4]/worst',
        'quiet': True,
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        video_url = info['url']
        fps = info.get('fps', 25)
        duration = info.get('duration', 0)
        if max_duration:
            duration = min(duration, max_duration)
    
    print(f"   视频时长: {duration}秒")
    print(f"   帧率: {fps} fps")
    
    # 计算采样间隔
    frame_interval = int(fps / sample_fps) if fps > 0 else 30
    
    # ffmpeg 命令：从URL读取视频，输出原始帧到stdout
    ffmpeg_cmd = [
        'ffmpeg',
        '-i', video_url,
        '-loglevel', 'error',
        '-t', str(duration),  # 只处理指定时长
        '-vf', f'select=not(mod(n\\,{frame_interval}))',  # 采样
        '-vsync', '0',
        '-f', 'image2pipe',
        '-pix_fmt', 'bgr24',
        '-vcodec', 'rawvideo',
        'pipe:'
    ]
    
    print(f"\n🔍 开始流式OCR...")
    print(f"   采样: 每 {frame_interval} 帧提取 1 帧")
    
    try:
        # 启动 ffmpeg 进程
        process = subprocess.Popen(
            ffmpeg_cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            bufsize=10**8
        )
        
        # 读取帧尺寸（需要先读取一帧获取）
        raw_bytes = process.stdout.read(640 * 480 * 3)  # 假设初始尺寸
        if not raw_bytes:
            print("⚠️ 无法读取视频流")
            return None
        
        # 简化：使用固定尺寸或从ffprobe获取
        import json
        probe_cmd = [
            'ffprobe', '-v', 'error', '-select_streams', 'v:0',
            '-show_entries', 'stream=width,height,r_frame_rate',
            '-of', 'json', video_url
        ]
        probe_result = subprocess.run(probe_cmd, capture_output=True, text=True)
        probe_data = json.loads(probe_result.stdout)
        width = int(probe_data['streams'][0]['width'])
        height = int(probe_data['streams'][0]['height'])
        
        frame_size = width * height * 3
        frame_idx = 0
        last_time = time.time()
        
        while True:
            # 读取一帧
            raw_frame = process.stdout.read(frame_size)
            
            if len(raw_frame) != frame_size:
                break
            
            # 转换为 numpy 数组
            frame = np.frombuffer(raw_frame, dtype=np.uint8).reshape((height, width, 3))
            
            # OCR 处理
            result = ocr.ocr(frame, cls=True)
            frame_count += 1
            
            current_time = frame_idx / fps
            
            # 提取文字
            if result and result[0]:
                for line in result[0]:
                    if line[1][1] >= conf_threshold:
                        all_texts.append({
                            'frame': frame_idx,
                            'time': current_time,
                            'text': line[1][0],
                            'conf': line[1][1]
                        })
                        text_count += 1
            
            frame_idx += frame_interval
            
            # 进度显示
            if frame_idx % 250 == 0:
                elapsed = time.time() - last_time
                progress = current_time / duration
                speed = (current_time / (time.time() - total_start))
                print(f"   进度: {progress*100:.1f}% ({current_time:.1f}s/{duration}s) | 速度: {speed:.2f}x实时")
                last_time = time.time()
        
        process.terminate()
        
    except Exception as e:
        print(f"⚠️ 流式处理中断: {e}")
        if 'process' in locals():
            process.terminate()
    
    total_time = time.time() - total_start
    
    print(f"\n✅ 流式处理完成!")
    print(f"   处理帧数: {frame_count}")
    print(f"   识别文字: {text_count} 条")
    print(f"   总耗时: {total_time:.2f}秒")
    print(f"   速度比: {duration/total_time:.2f}x 实时")
    
    return {
        'method': 'streaming_ocr',
        'frame_count': frame_count,
        'text_count': text_count,
        'video_duration': duration,
        'total_time': total_time,
        'speed_ratio': duration / total_time,
        'texts': all_texts
    }

# 初始化 OCR
print("\n📦 初始化 PaddleOCR...")
ocr_start = time.time()
ocr = PaddleOCR(use_angle_cls=True, lang='ch', use_gpu=gpu_available, show_log=False)
print(f"   模型加载耗时: {time.time()-ocr_start:.2f}秒")

# 运行流式 OCR（只处理前60秒以加快测试）
result = stream_video_with_ocr(
    TEST_URL,
    ocr=ocr,
    sample_fps=SAMPLE_FPS,
    conf_threshold=CONF_THRESHOLD,
    max_duration=60  # 只处理前60秒
)

# 显示结果
if result and result['texts']:
    print("\n" + "="*60)
    print("📝 识别到的文字:")
    print("="*60)
    
    unique_texts = []
    last_text = None
    for item in sorted(result['texts'], key=lambda x: x['time']):
        text = item['text'].strip()
        if text and text != last_text:
            unique_texts.append(item)
            last_text = text
    
    for i, item in enumerate(unique_texts[:15], 1):
        print(f"[{i}] {item['time']:.1f}s: {item['text']}")

## 3️⃣ OCR 性能测试函数

In [ ]:
def test_ocr_performance(video_path, use_gpu=True, lang='ch',
                         sample_fps=1, conf_threshold=0.5):
    """
    测试 OCR 性能
    
    Args:
        video_path: 视频文件路径
        use_gpu: 是否使用 GPU
        lang: 语言 ('ch'中文, 'en'英文)
        sample_fps: 采样帧率 (每秒提取多少帧)
        conf_threshold: 置信度阈值
    
    Returns:
        性能数据字典
    """
    device = 'GPU' if use_gpu else 'CPU'
    print(f"\n{'='*60}")
    print(f"🧪 {device} 模式测试")
    print(f"{'='*60}")
    
    # 初始化 OCR
    init_start = time.time()
    ocr = PaddleOCR(
        use_angle_cls=True,
        lang=lang,
        use_gpu=use_gpu,
        show_log=False
    )
    init_time = time.time() - init_start
    print(f"模型加载: {init_time:.2f}秒")
    
    # 打开视频
    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_duration = total_frames / fps
    
    # 计算采样间隔
    frame_interval = int(fps / sample_fps) if fps > 0 else 30
    sample_count = int(total_frames / frame_interval)
    
    print(f"视频帧率: {fps:.1f} fps")
    print(f"总帧数: {total_frames}")
    print(f"采样间隔: 每 {frame_interval} 帧提取 1 帧")
    print(f"预计采样: {sample_count} 帧")
    
    # OCR 处理
    ocr_start = time.time()
    all_texts = []
    frame_count = 0
    text_count = 0
    
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # 采样
        if frame_idx % frame_interval == 0:
            result = ocr.ocr(frame, cls=True)
            frame_count += 1
            
            # 提取文字
            if result and result[0]:
                for line in result[0]:
                    if line[1][1] >= conf_threshold:
                        all_texts.append({
                            'frame': frame_idx,
                            'time': frame_idx / fps,
                            'text': line[1][0],
                            'conf': line[1][1]
                        })
                        text_count += 1
        
        frame_idx += 1
        
        # 进度显示
        if frame_idx % 1000 == 0:
            elapsed = time.time() - ocr_start
            progress = frame_idx / total_frames
            print(f"进度: {progress*100:.1f}% ({frame_idx}/{total_frames}帧)")
    
    cap.release()
    ocr_time = time.time() - ocr_start
    
    print(f"\n✅ 处理完成!")
    print(f"   处理帧数: {frame_count}")
    print(f"   识别文字: {text_count} 条")
    print(f"   OCR 耗时: {ocr_time:.2f}秒")
    print(f"   速度比: {video_duration/ocr_time:.2f}x 实时")
    print(f"   平均每帧: {ocr_time/frame_count*1000:.1f}ms")
    
    return {
        'device': device,
        'lang': lang,
        'init_time': init_time,
        'ocr_time': ocr_time,
        'total_time': init_time + ocr_time,
        'frame_count': frame_count,
        'text_count': text_count,
        'video_duration': video_duration,
        'speed_ratio': video_duration / ocr_time,
        'avg_ms_per_frame': ocr_time / frame_count * 1000 if frame_count > 0 else 0,
        'texts': all_texts
    }

## 4️⃣ CPU vs GPU 速度对比

In [ ]:
# 测试参数
SAMPLE_FPS = 1  # 每秒提取 1 帧（提高速度）
CONF_THRESHOLD = 0.5

results = {}

# ============ CPU 测试 ============
print("\n🖥️  CPU 模式")
results['cpu'] = test_ocr_performance(
    video_file, 
    use_gpu=False,
    lang='ch',
    sample_fps=SAMPLE_FPS,
    conf_threshold=CONF_THRESHOLD
)

# ============ GPU 测试 ============
if gpu_available:
    print("\n🚀 GPU 模式")
    results['gpu'] = test_ocr_performance(
        video_file, 
        use_gpu=True,
        lang='ch',
        sample_fps=SAMPLE_FPS,
        conf_threshold=CONF_THRESHOLD
    )
else:
    print("\n⚠️ GPU 不可用，跳过 GPU 测试")

## 5️⃣ 测试结果对比

In [ ]:
print("\n" + "="*60)
print("📊 测试结果对比")
print("="*60)

print(f"\n视频时长: {results['cpu']['video_duration']:.1f}秒")
print(f"采样帧率: {SAMPLE_FPS} fps")
print(f"置信度阈值: {CONF_THRESHOLD}")

if 'cpu' in results:
    r = results['cpu']
    print(f"\n🖥️  CPU:")
    print(f"   模型加载: {r['init_time']:.2f}秒")
    print(f"   OCR 耗时: {r['ocr_time']:.2f}秒")
    print(f"   总耗时: {r['total_time']:.2f}秒")
    print(f"   速度比: {r['speed_ratio']:.2f}x 实时")
    print(f"   平均每帧: {r['avg_ms_per_frame']:.1f}ms")
    print(f"   识别文字: {r['text_count']} 条")

if 'gpu' in results:
    r = results['gpu']
    print(f"\n🚀 GPU:")
    print(f"   模型加载: {r['init_time']:.2f}秒")
    print(f"   OCR 耗时: {r['ocr_time']:.2f}秒")
    print(f"   总耗时: {r['total_time']:.2f}秒")
    print(f"   速度比: {r['speed_ratio']:.2f}x 实时")
    print(f"   平均每帧: {r['avg_ms_per_frame']:.1f}ms")
    print(f"   识别文字: {r['text_count']} 条")
    
    # 计算加速比
    if 'cpu' in results:
        speedup = results['cpu']['ocr_time'] / results['gpu']['ocr_time']
        time_saved = results['cpu']['ocr_time'] - results['gpu']['ocr_time']
        
        print(f"\n🏆 GPU 加速效果:")
        print(f"   OCR 加速: {speedup:.2f}x")
        print(f"   时间节省: {time_saved:.2f}秒 ({time_saved/results['cpu']['ocr_time']*100:.1f}%)")
        
        # 推算长视频时间
        print(f"\n📈 长视频耗时推算:")
        for video_len in [5, 10, 30]:  # 分钟
            seconds = video_len * 60
            cpu_time = seconds / results['cpu']['speed_ratio']
            gpu_time = seconds / results['gpu']['speed_ratio']
            print(f"   {video_len}分钟视频: CPU {cpu_time/60:.1f}分钟 vs GPU {gpu_time/60:.1f}分钟")

## 6️⃣ 查看识别结果

In [ ]:
# 显示识别到的文字（去重后）
source = results['gpu'] if 'gpu' in results else results['cpu']

print("\n" + "="*60)
print("📝 识别到的文字 (按时间顺序)")
print("="*60)

if source['texts']:
    # 按时间排序并去重（相邻相同文本合并）
    unique_texts = []
    last_text = None
    
    for item in sorted(source['texts'], key=lambda x: x['time']):
        text = item['text'].strip()
        if text and text != last_text:
            unique_texts.append({
                'time': item['time'],
                'text': text,
                'conf': item['conf']
            })
            last_text = text
    
    print(f"\n共识别到 {len(unique_texts)} 条不重复文字:\n")
    
    for i, item in enumerate(unique_texts[:20], 1):  # 显示前20条
        print(f"[{i}] {item['time']:.1f}s ({item['conf']:.2f}): {item['text']}")
    
    if len(unique_texts) > 20:
        print(f"\n... 还有 {len(unique_texts)-20} 条")
else:
    print("\n⚠️ 未识别到任何文字")
    print("   可能原因:")
    print("   - 视频中没有文字")
    print("   - 置信度阈值过高")
    print("   - 采样帧率过低")

## 7️⃣ 不同采样率性能测试（可选）

In [ ]:
# 测试不同采样率的影响（仅 GPU）
if gpu_available:
    sample_results = {}
    
    for fps in [0.5, 1, 2, 5]:
        print(f"\n测试采样率: {fps} fps")
        result = test_ocr_performance(
            video_file,
            use_gpu=True,
            sample_fps=fps,
            conf_threshold=CONF_THRESHOLD
        )
        sample_results[fps] = result
    
    # 对比
    print("\n" + "="*60)
    print("📊 采样率对比")
    print("="*60)
    print(f"\n{'采样率':<8} {'处理帧数':<10} {'OCR耗时':<10} {'识别文字':<10} {'速度比':<10}")
    print("-"*60)
    
    for fps, r in sample_results.items():
        print(f"{fps:<8.1f} {r['frame_count']:<10} {r['ocr_time']:<10.1f} {r['text_count']:<10} {r['speed_ratio']:<10.2f}x")

## 📝 预期结果

### PaddleOCR GPU vs CPU 性能对比 (Colab T4 GPU):

| 项目 | CPU | GPU | 加速比 |
|------|-----|-----|--------|
| 模型加载 | ~3秒 | ~2秒 | 1.5x |
| 单帧处理 | ~200-500ms | ~20-50ms | **5-10x** |
| 速度比 | ~0.1x 实时 | ~0.5-1x 实时 | **5-10x** |

### 典型场景推算 (中文 OCR, 每秒采样 1 帧):

| 视频时长 | CPU 耗时 | GPU 耗时 |
|----------|----------|----------|
| 5分钟 | ~15-30分钟 | ~2-5分钟 |
| 10分钟 | ~30-60分钟 | ~5-10分钟 |
| 30分钟 | ~1.5-3小时 | ~15-30分钟 |

### 💡 优化建议:

1. **提高采样率**: 对于字幕密集视频，使用 2-5 fps
2. **降低置信度**: 使用 0.3-0.5 阈值捕获更多文字
3. **ROI 裁剪**: 如果字幕位置固定，先裁剪字幕区域
4. **使用 GPU**: GPU 加速效果显著，强烈推荐